In [2]:
import math
import os
import shutil

In [3]:
def parse_problem(problem_file):

    cx_obs = []
    cy_obs = []
    t_obs = []

    with open(problem_file, "r") as f:
        for line in f.readlines():
            if "(theta)" in line:
                init_theta = float(line.strip().replace("(", "").replace(")", "").split(" ")[2])
            elif "(v)" in line:
                init_v = float(line.strip().replace("(", "").replace(")", "").split(" ")[2])
            elif "(time_obs" in line:
                t_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(cx_obs" in line:
                cx_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(cy_obs" in line:
                cy_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]


    observations = list(zip(cx_obs, cy_obs, t_obs))
    
    return observations, init_theta, init_v

In [24]:
def define_ha(observations, init_theta, init_v):
    
    num_obs = len(observations)
    lower_bound = -2000
    upper_bound = 2000
    
    ha_str = ""
        
    # Variables
    ha_str += "[0,{}] time;\n".format(observations[-1][2] + 0.05)
    ha_str += "[{},{}] x1;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x2;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] theta;\n".format(-100,100)
    ha_str += "[0,{}] clock;\n".format(observations[-1][2] + 0.05)
    ha_str += "\n"
    
    
    # Modes
    modes = {}
    num_modes = 0
    for hmode in ["flying_straight", "adjusting_left", "adjusting_right"]:
        for obs_index in range(num_obs+1):
            modes[(hmode,obs_index)] = num_modes + 1
            num_modes += 1
    
    for hmode in ["flying_straight", "adjusting_left", "adjusting_right"]:
            
        for obs_index in range(num_obs+1):
            mode_id = modes[(hmode, obs_index)]
            ha_str += "// {}_obs{}\n".format(hmode,obs_index)
            ha_str += "{\n"
            ha_str += "\t mode {};\n".format(mode_id)
            # Invariant
            ha_str += "\t invt:\n"
            if obs_index < num_obs:
                ha_str += "\t\t (clock < {} + 0.05);\n".format(observations[obs_index][2])
            else:
                ha_str += "\t\t (clock < {} + 0.05);\n".format(observations[-1][2])
            # Flow
            ha_str += "\t flow:\n"
            ha_str += "\t\t d/dt[x1] = {} * cos(theta);\n".format(init_v)
            ha_str += "\t\t d/dt[x2] = {} * sin(theta);\n".format(init_v)
                       
            if hmode == "flying_straight":
                ha_str += "\t\t d/dt[theta] = 0;\n"
            elif hmode == "adjusting_left":
                ha_str += "\t\t d/dt[theta] = 0.31415;\n"
            else:
                ha_str += "\t\t d/dt[theta] = -0.31415;\n"
                
            ha_str += "\t\t d/dt[clock] = 1.0;\n"
            
            # Jumps
            
            ha_str += "\t jump:\n"
            if hmode == "flying_straight":
                ha_str += "\t\t (true) ==> @{}(and (x1' = x1) (x2' = x2) (theta' = theta) (clock' = clock));\n".format(modes[("adjusting_left", obs_index)])
                ha_str += "\t\t (true) ==> @{}(and (x1' = x1) (x2' = x2) (theta' = theta) (clock' = clock));\n".format(modes[("adjusting_right", obs_index)])
            elif hmode == "adjusting_left":
                ha_str += "\t\t (true) ==> @{}(and (x1' = x1) (x2' = x2) (theta' = theta) (clock' = clock));\n".format(modes[("flying_straight", obs_index)])
                ha_str += "\t\t (true) ==> @{}(and (x1' = x1) (x2' = x2) (theta' = theta) (clock' = clock));\n".format(modes[("adjusting_right", obs_index)])
            else:
                ha_str += "\t\t (true) ==> @{}(and (x1' = x1) (x2' = x2) (theta' = theta) (clock' = clock));\n".format(modes[("adjusting_left", obs_index)])
                ha_str += "\t\t (true) ==> @{}(and (x1' = x1) (x2' = x2) (theta' = theta) (clock' = clock));\n".format(modes[("flying_straight", obs_index)])
            
            if obs_index < num_obs:    
                ha_str += "\n"
                ha_str += "\t\t (and (clock > {clock_obs} - 0.05) (clock < {clock_obs} + 0.05) ((x1 - {x1_obs})^2 + (x2 - {x2_obs})^2 < 400)) ==> @{dst_id}(and (x1' = x1) (x2' = x2) (theta' = theta) (clock' = clock));\n".format(clock_obs = observations[obs_index][2], x1_obs = observations[obs_index][0], x2_obs = observations[obs_index][1], dst_id = modes[(hmode, obs_index+1)])
            ha_str += "}\n"
            
    # Init
    ha_str += "init:\n"
    ha_str += "@1 (and (x1 = 0.0) (x2 = 0.0) (theta = {}) (clock = 0.0));\n".format(init_theta)
    ha_str += "\n"
    
    # Goal
    ha_str += "goal:\n"
    for hmode in ["flying_straight", "adjusting_left", "adjusting_right"]:
        mode_id = modes[(hmode, num_obs)]
        ha_str += "@{} true;\n".format(mode_id)
            
    return ha_str   

In [26]:
num_instances = 10
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

dfolder = "benchmarks/flight/dreach"
if os.path.exists(dfolder):
    shutil.rmtree(dfolder)
os.makedirs(dfolder)


for pnum in range(num_instances):
    for h in horizons:
        pname = 'inst_{pnum}_{h}.drh'.format(pnum=pnum, h=h)
        
        observations, init_theta, init_v = parse_problem("benchmarks/flight/generation/inst_{}_{}_20_100.pddl".format(pnum, h))

        ha_str = define_ha(observations, init_theta, init_v)
        with open(dfolder + '/' + pname, 'w') as f:
            f.write(ha_str)